In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.5.1


#### Importing libraries:

In [2]:
import torch

#### Computing self-attention with regards to particular token:

In [3]:
inputs  = torch.tensor([
   [0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
query = inputs[1] #2nd input token is query

attention_scores_2 = torch.empty(inputs.shape[0])
for i,x_i in enumerate(inputs):
    print(f"Index:{i} \t Input_token:{x_i} ")
    attention_scores_2[i] = torch.dot(x_i,query)

print(attention_scores_2)

Index:0 	 Input_token:tensor([0.4300, 0.1500, 0.8900]) 
Index:1 	 Input_token:tensor([0.5500, 0.8700, 0.6600]) 
Index:2 	 Input_token:tensor([0.5700, 0.8500, 0.6400]) 
Index:3 	 Input_token:tensor([0.2200, 0.5800, 0.3300]) 
Index:4 	 Input_token:tensor([0.7700, 0.2500, 0.1000]) 
Index:5 	 Input_token:tensor([0.0500, 0.8000, 0.5500]) 
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
attention_weights_2_tmp = attention_scores_2 / attention_scores_2.sum()
print(f"Attention weights:{attention_weights_2_tmp}")
print(f"Sum:{attention_weights_2_tmp.sum()}")

Attention weights:tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:1.0000001192092896


In [6]:
def softmax_naive(x):
    """
    this might result in problems like numerical underflow and overflow
    therefor better to use Pytorch's inbuilt softmax function
    """
    return torch.exp(x)/torch.exp(x).sum(dim=0)

attention_weights_2_naive = softmax_naive(attention_scores_2)
print(f"Attention weights:{attention_weights_2_naive}")
print(f"Sum:{attention_weights_2_naive.sum()}")

Attention weights:tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:1.0


In [7]:
attention_weights_2 = torch.softmax(attention_scores_2,dim=0)
print(f"Attention weights:{attention_weights_2}")
print(f"Sum:{attention_weights_2.sum()}")

Attention weights:tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:1.0


In [8]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attention_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


#### Computing attention weights for all input tokens(naive approach):

In [9]:
attention_scores = inputs@inputs.T # alternative to using for loops
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
attention_weights = torch.softmax(attention_scores,dim=-1) 
#dim =-1 suggests the normalization along last dimention of attention_scores tensor
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [11]:
all_context_vecs = attention_weights@inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [12]:
#summing up all  process into single function

def self_attention_naive(inputs):
    """
    Implements a simple naive self-attention mechanism without using
    any trainable weights.
    The process consists of three operations:
    - Matmul (query and input tokens)
    - Normalization (normalize the scores obtained previous)
    - Matmul (context vectors obtained)
    """
    attention_scores = inputs @ inputs.T
    normalized_attention_scores = torch.softmax(attention_scores,dim=-1)
    context_vectors = normalized_attention_scores @ inputs 
    return context_vectors

self_attention_naive(inputs)


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

#### Implementing self-attention with trainable weights(one vector):

In [13]:
x_2 = inputs[1]
d_in = inputs.shape[1] # input embedding size
d_out = 2 # output embedding size 

In [14]:
torch.manual_seed(42)

In [15]:
W_query = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [16]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(f"Query_2 : {query_2}")

Query_2 : tensor([1.0760, 1.7344])


In [17]:
# for all input elements
key = inputs @ W_key
values = inputs @ W_value
print(f"keys shape: {key.shape}")
print(f"Values shape:{values.shape}")

keys shape: torch.Size([6, 2])
Values shape:torch.Size([6, 2])


In [18]:
# attention score for 2 vector

keys_2 = key[1]
attention_scores_22 = query_2.dot(keys_2)
print(attention_scores_22)

tensor(3.3338)


In [19]:
# generalized attention scores

attention_scores_2 = query_2 @ key.T
print(attention_scores_2)

tensor([2.7084, 3.3338, 3.3013, 1.7563, 1.7869, 2.1966])


In [20]:
d_k =  key.shape[-1]
attention_weights_2 = torch.softmax(attention_scores_2/d_k** 0.5 , dim=-1)
print(attention_weights_2)

tensor([0.1723, 0.2681, 0.2620, 0.0879, 0.0898, 0.1200])


In [29]:
context_vec_2 = attention_weights_2 @ values
print(context_vec_2)

tensor([1.4201, 0.8892])


#### Generalized self-attention code:

In [22]:
import torch.nn as nn

In [36]:
class SelfAttentionV1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))

    def forward(self,x):
        query = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value
        attention_scores = query @ keys.T
        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]** 0.5 , dim = -1
        )
        context_vectors = attention_weights @ values
        return context_vectors


In [37]:
torch.manual_seed(42)

In [38]:
self_attention_v1 = SelfAttentionV1(d_in,d_out)
print(self_attention_v1(inputs))

tensor([[1.3751, 0.8610],
        [1.4201, 0.8892],
        [1.4198, 0.8890],
        [1.3533, 0.8476],
        [1.3746, 0.8606],
        [1.3620, 0.8532]], grad_fn=<MmBackward0>)


In [39]:
class SelfAttentionV2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias= False):
        super().__init__()
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)

    def forward(self,x):
        queries = self.W_query(x)
        keys =  self.W_key(x)
        values = self.W_value(x)
        attention_scores = queries @ keys.T
        attention_weights = torch.softmax( 
            attention_scores / keys.shape[-1] ** 0.5 , dim =-1
            )
        context_vectors = attention_weights @ values
        return context_vectors

In [40]:
torch.manual_seed(42)

self_attention_v2 = SelfAttentionV2(d_in,d_out)
print(self_attention_v2(inputs))

tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)


- The difference in the results of V1 and V2 is due to different weight initialization strategy

#### Implementing simple causal attention mask 

In [41]:
queries = self_attention_v2.W_query(inputs)
keys = self_attention_v2.W_key(inputs)
attention_scores = queries @ keys.T
attention_weights = torch.softmax(attention_scores / keys.shape[-1]** 0.5 , dim = -1)
print(attention_weights)

tensor([[0.1605, 0.1726, 0.1714, 0.1681, 0.1473, 0.1801],
        [0.1627, 0.1780, 0.1758, 0.1648, 0.1306, 0.1880],
        [0.1625, 0.1782, 0.1759, 0.1648, 0.1302, 0.1885],
        [0.1661, 0.1726, 0.1715, 0.1654, 0.1475, 0.1768],
        [0.1596, 0.1777, 0.1755, 0.1664, 0.1312, 0.1896],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<SoftmaxBackward0>)


In [43]:
# masking of the weights
context_length = attention_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [44]:
masked_simple = attention_weights * mask_simple
print(masked_simple)

tensor([[0.1605, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1627, 0.1780, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1625, 0.1782, 0.1759, 0.0000, 0.0000, 0.0000],
        [0.1661, 0.1726, 0.1715, 0.1654, 0.0000, 0.0000],
        [0.1596, 0.1777, 0.1755, 0.1664, 0.1312, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<MulBackward0>)


In [45]:
row_sums = masked_simple.sum(dim=-1,keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4775, 0.5225, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3146, 0.3450, 0.3405, 0.0000, 0.0000, 0.0000],
        [0.2459, 0.2555, 0.2538, 0.2448, 0.0000, 0.0000],
        [0.1969, 0.2193, 0.2165, 0.2053, 0.1619, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<DivBackward0>)


#### Implementation of more efficient causal masked attention:

- Here we would be masking the attention scores with negative inf values before
applying the softmax function

In [49]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked = attention_scores.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[ 0.0508,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.2157,  0.3428,    -inf,    -inf,    -inf,    -inf],
        [ 0.2163,  0.3467,  0.3282,    -inf,    -inf,    -inf],
        [ 0.1257,  0.1799,  0.1707,  0.1191,    -inf,    -inf],
        [ 0.1667,  0.3193,  0.3012,  0.2258, -0.1098,    -inf],
        [ 0.1269,  0.1548,  0.1475,  0.0978, -0.0247,  0.1731]],
       grad_fn=<MaskedFillBackward0>)


In [55]:
attention_weights = torch.softmax(masked/keys.shape[-1]**0.5 , dim = 1)
print(attention_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4775, 0.5225, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3146, 0.3450, 0.3405, 0.0000, 0.0000, 0.0000],
        [0.2459, 0.2555, 0.2538, 0.2448, 0.0000, 0.0000],
        [0.1969, 0.2193, 0.2165, 0.2053, 0.1619, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<SoftmaxBackward0>)


- After this step , the rest of the process remains the same.
- We get to compute the context vectors by (attention_weights@values)

#### Usage of droput:

In [51]:
torch.manual_seed(42)

In [52]:
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[0., 0., 2., 2., 2., 2.],
        [2., 0., 2., 0., 2., 0.],
        [0., 0., 2., 2., 2., 0.],
        [2., 2., 0., 2., 0., 2.],
        [2., 0., 2., 2., 2., 2.],
        [2., 2., 2., 0., 2., 0.]])


- The values in the above matrix equals 2, cause to compensate for the reduction of the active elements in the matrix.
- They are scaled by a factor of (1/0.5=2).

In [53]:
torch.manual_seed(42)

In [56]:
print(dropout(attention_weights))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9551, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6292, 0.0000, 0.6809, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5110, 0.0000, 0.4895, 0.0000, 0.0000],
        [0.3938, 0.4387, 0.4331, 0.0000, 0.0000, 0.0000],
        [0.3364, 0.0000, 0.0000, 0.0000, 0.0000, 0.3476]],
       grad_fn=<MulBackward0>)


#### Generalized causal attention:

In [57]:
batch = torch.stack((inputs,inputs),dim=0) # handling batches of more than one input
print(batch.shape)

torch.Size([2, 6, 3])


In [58]:
class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,
                 dropout,qkv_bias= False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length,context_length),
            diagonal=1)
        )


    def forward(self,x):
        b , num_tokens , d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attention_scores = queries @ keys.transpose(1,2) # b , d_in , num_tokens
        attention_scores.masked_fill_(  # helps in avoiding unnecessary memory copies
            self.mask.bool()[:num_tokens,:num_tokens],-torch.inf
        )
        attention_weights = torch.softmax(
            attention_scores/keys.shape[-1]**0.5 ,dim =-1
        )
        attention_weights = self.dropout(attention_weights)
        context_vectors = attention_weights @ values 
        return context_vectors


Advantages of using register buffer:
- buffers are automatically moved to the appropriate device (GPU/CPU)
    - doesn't require manual conversion of tensors to CPU / GPU 

In [59]:
torch.manual_seed(42)

In [60]:
context_length = batch.shape[1]
casual_attention = CausalAttention(d_in,d_out,context_length,0.0)
context_vec = casual_attention(batch)
print(f"context_vecs.shape:{context_vec.shape}")

context_vecs.shape:torch.Size([2, 6, 2])


#### Simple Multihead attention:
- Making attention mechanism run individually but then separately (heads)

In [61]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in,d_out,context_length,
                 dropout,num_heads,qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in,d_out,context_length,dropout,qkv_bias
            )
            for _ in range(num_heads)
            ]
        )
    
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)

In [62]:
torch.manual_seed(42)

In [66]:
context_length = batch.shape[1]
d_in , d_out = 3 , 1
multi_head_attention = MultiHeadAttentionWrapper(
    d_in,d_out,context_length,0.0,num_heads=2
)
context_vectors = multi_head_attention(batch)
print(context_vectors)
print(f"shape of context vectors:{context_vectors.shape}") #no_of_input_text, no_tokens, dim_embedding

tensor([[[-0.0836, -0.4057],
         [-0.2409, -0.2476],
         [-0.2906, -0.2040],
         [-0.2831, -0.1717],
         [-0.2538, -0.1165],
         [-0.2722, -0.1182]],

        [[-0.0836, -0.4057],
         [-0.2409, -0.2476],
         [-0.2906, -0.2040],
         [-0.2831, -0.1717],
         [-0.2538, -0.1165],
         [-0.2722, -0.1182]]], grad_fn=<CatBackward0>)
shape of context vectors:torch.Size([2, 6, 2])


#### Efficient Multi head attention:

- The earlier code involved using a wrapper that called Causal attention
sequentially
- Instead we condense it into a single function aiming to split the process parallely

In [67]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,
                 context_length,dropout,
                 num_heads, qkv_bias=False):
        
        super().__init__()
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.out_proj = nn.Linear(d_out,d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length),
                       diagonal=1)
        )


    def forward(self,x):
        b, num_tokens , d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Splitting the matrix by adding num_heads
        keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)
        values = values.view(b,num_tokens,self.num_heads,self.head_dim)


        # Converting the shape
        # b,num_tokens,num_heads,head_dim = b,num_heads,num_tokens,head_dim
        # crucial for aligning the keys and values in multiple heads
        keys = keys.transpose(1,2) 
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)

        attention_scores = queries @ keys.transpose(2,3) # dot product for each head
        mask_bool = self.mask.bool()[:num_tokens,:num_tokens]
        attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(
            attention_scores / keys.shape[-1]**0.5 , dim = -1
        )
        attention_weights = self.dropout(attention_weights)

        # b , num_token , n_heads , head_dim
        context_vectors = (attention_weights @ values).transpose(1,2) 

        # combines the heads 
        context_vectors = context_vectors.contiguous().view(
            b,num_tokens,self.d_out
        )
        context_vectors = self.out_proj(context_vectors)
        return context_vectors

In [68]:
torch.manual_seed(42)

In [69]:
batch_size , context_length , d_in = batch.shape
d_out = 2
multi_head_attention = MultiHeadAttention(d_in,d_out,context_length,0.0,num_heads=2)
context_vectors = multi_head_attention(batch)
print(context_vectors)
print(f"context vectors shape:{context_vectors.shape}")

tensor([[[-0.4140,  0.3158],
         [-0.3942,  0.2961],
         [-0.3888,  0.2900],
         [-0.3714,  0.3100],
         [-0.3764,  0.3150],
         [-0.3632,  0.3238]],

        [[-0.4140,  0.3158],
         [-0.3942,  0.2961],
         [-0.3888,  0.2900],
         [-0.3714,  0.3100],
         [-0.3764,  0.3150],
         [-0.3632,  0.3238]]], grad_fn=<ViewBackward0>)
context vectors shape:torch.Size([2, 6, 2])
